In [ ]:
from IPython.display import Image
Image(filename='prometheus.png',width=150, height=150)

# Analyze Prometheus Logs

In [ ]:
S3_LOGS_URL = ''

In [ ]:
# verify that S3_LOGS_URL is not empty
import sys
from IPython.display import display, Javascript

if not S3_LOGS_URL:
    display(Javascript('alert("Enter S3_LOGS_URL, click >> button")'))
    sys.exit()

## Prerequsite 

In [ ]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

# extract credentials
import configparser
config = configparser.ConfigParser()
config.read('logs.conf')
username = config.get("data", 'user') 
password = config.get("data", 'password')
grafana_dashboard_url = config.get("data", 'grafana_dashboard_url')


# Update fedora
!dnf update -y

# Logs operations
from benchmark_runner.jupyterlab.templates.logs_operations.logs_operations import LogsOperations
logs_operations = LogsOperations(s3_logs_url=S3_LOGS_URL)
# Logs cleanup
logs_operations.cleanup()

# Import AnalyzePrometheusLogs class and initialized
from benchmark_runner.jupyterlab.templates.analyze_prometheus_logs.analyze_prometheus_logs import AnalyzePrometheusLogs
analyze_prometheus_logs = AnalyzePrometheusLogs(s3_logs_url=S3_LOGS_URL)


## Download S3 logs

In [ ]:
logs_operations.download_s3_logs(username=username, password=password)

## Untar prometheus logs & chmod 

In [ ]:
promdb_dir_path = analyze_prometheus_logs.untar_and_chmod_prometheus_logs()

## Run Prometheus container 

In [ ]:
analyze_prometheus_logs.run_container(image_name='prometheus', command=f"-d -uroot --ipc=host --pid=host --net=host -v /sys:/sys -v /proc:/proc -v /dev:/dev -v {promdb_dir_path}:/prometheus --privileged docker.io/prom/prometheus:latest --config.file=/etc/prometheus/prometheus.yml --storage.tsdb.path=/prometheus --storage.tsdb.retention.time=100000d --storage.tsdb.retention.size=1000GB")

## Result: Grafana direct link

In [ ]:
analyze_prometheus_logs.open_grafana_dashboard(promdb_dir_path=promdb_dir_path, grafana_dashboard_url=grafana_dashboard_url)